* Author: Yi-Shin Lin and others
* Date: 14/08/2023
* License: GPL3
* Title: Fitting the data subset of the AGE scenario

In [2]:
import Model
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from IPython.display import Image
import Model_utility 
import pickle
import os
os.chdir('/media/yslin/kakapu/02_Projects/Multi-choices_Github/')

In [5]:
# Load res1_list from a file
with open('data/result0.pkl', 'rb') as f:
    result = pickle.load(f)
    
print(result)

       message: Maximum number of iterations has been exceeded.
       success: False
        status: 2
           fun: 3854.236074375918
             x: [ 4.500e-01  4.750e-01  5.500e-01]
           nit: 3
          nfev: 8
 final_simplex: (array([[ 4.500e-01,  4.750e-01,  5.500e-01],
                       [ 4.500e-01,  5.250e-01,  5.250e-01],
                       [ 5.000e-01,  5.000e-01,  5.250e-01],
                       [ 5.000e-01,  5.000e-01,  5.000e-01]]), array([ 3.854e+03,  3.868e+03,  3.879e+03,  3.888e+03]))


In [6]:
# Load res1_list from a file
with open('data/result1.pkl', 'rb') as f:
    result = pickle.load(f)
    
print(result)

       message: Maximum number of iterations has been exceeded.
       success: False
        status: 2
           fun: 2734.46196464687
             x: [ 4.500e-01  4.750e-01  5.500e-01]
           nit: 3
          nfev: 8
 final_simplex: (array([[ 4.500e-01,  4.750e-01,  5.500e-01],
                       [ 4.500e-01,  5.250e-01,  5.250e-01],
                       [ 5.000e-01,  5.000e-01,  5.250e-01],
                       [ 5.000e-01,  5.000e-01,  5.000e-01]]), array([ 2.734e+03,  2.744e+03,  2.750e+03,  2.756e+03]))


In [2]:
def objective_function(x, RT, R, filename):
    drift0 = x[0]
    start_point = x[1]
    ndt = x[2]

    bound0 = 3.0  # Fixed bound0 at 3.0. You should run a grid search to tune this parameter to find the best case that fit the data.
    noise0 = 2.5
    dx = 0.01
    dt = 0.01
    max_time = 14.0

    bias = "point"
    method = "implicit"
    
    estimated_pdf = Model.test_solve_numerical(method, bias, drift0, bound0, noise0, start_point, dx, dt, max_time)

    # RT and R were passed on as global variables. I have not found a way to
    # pass them on as arguments to the objective function. 
    # TODO: look up the scipy.minimize document to see if there is a way to
    # that.
    res = Model_utility.calculate_LL(RT, R, estimated_pdf, dt, ndt)
    # Write the value of res to a file, so we can track the progress of 
    # the optimization to see if it converges to a stable value.
    with open(filename, 'a') as f:
        f.write(str(res) + '\n')

    return res
    

def callback(xk, filename):
    # ".join(map(str, xk))" is to remove the square brackets from the array.
    with open(filename, 'a') as f:
        f.write('   '.join(map(str, xk)) + '\n')
    # Print the estimate to the screen so we can also see its progress.
    # Comment it out, if you run the code on a cluster.
    print(xk)

In [3]:
df = pd.read_csv('data/age_text1.csv')
RT = df['RT'].to_numpy()
R = df['R'].to_numpy()
df.head()

s     RT     M Scenario_Type  R  A
0  1  5.664  Text           AGE  1  1
1  1  1.898  Text           AGE  1  1
2  1  3.164  Text           AGE  1  1
3  1  1.730  Text           AGE  1  1
4  1  2.163  Text           AGE  1  1

In [5]:
# In this simple case, it usually converges before 100 iterations
# Initial guess for the parameters. I renamed it to avoid using the same 
# variable name as the starting point.
initial_guess = [0.5, 0.5, 0.5]
filename1 = 'data/likelihood.csv'
filename2 = 'data/estimate_parameters.csv'
res1 = minimize(objective_function, initial_guess, (RT, R, filename1), method='Nelder-Mead', options={'maxiter': 3}, callback=lambda xk: callback(xk, filename2))

[0.45  0.525 0.525]
[0.45  0.475 0.55 ]


In [13]:
df = pd.read_csv('data/age_text2.csv')
RT = df['RT'].to_numpy()
R = df['R'].to_numpy()
df.head()

s     RT     M Scenario_Type  R  A
0  1  2.814  Text           AGE  1  2
1  1  2.297  Text           AGE  1  2
2  1  2.447  Text           AGE  1  2
3  1  1.014  Text           AGE  1  2
4  2  4.647  Text           AGE  1  2

In [14]:
# Rename the output likelihood_output.txt and estimate_output.txt in the data folder before running the code.
res2 = minimize(objective_function, initial_guess, method='Nelder-Mead', options={'maxiter': 150}, callback=callback)

[0.45  0.525 0.525]
[0.45  0.475 0.55 ]
[0.4 0.5 0.6]
[0.3   0.5   0.625]
[0.25  0.425 0.725]
[0.05  0.475 0.85 ]
[-0.2  0.4  1. ]
[-0.23333333  0.36666667  1.09166667]
[-0.50555556  0.40277778  1.23611111]
[-0.50555556  0.40277778  1.23611111]
[-0.50555556  0.40277778  1.23611111]
[-0.50555556  0.40277778  1.23611111]
[-0.44382716  0.36635802  1.2154321 ]
[-0.44382716  0.36635802  1.2154321 ]
[-0.44382716  0.36635802  1.2154321 ]
[-0.4361897   0.38975123  1.19777806]
[-0.4361897   0.38975123  1.19777806]
[-0.4361897   0.38975123  1.19777806]
[-0.41751687  0.3748695   1.19372578]
[-0.41751687  0.3748695   1.19372578]
[-0.41751687  0.3748695   1.19372578]
[-0.41751687  0.3748695   1.19372578]
[-0.41751687  0.3748695   1.19372578]
[-0.40221286  0.37400285  1.1845522 ]
[-0.40221286  0.37400285  1.1845522 ]
[-0.40221286  0.37400285  1.1845522 ]
[-0.40221286  0.37400285  1.1845522 ]
[-0.40221286  0.37400285  1.1845522 ]
[-0.40221286  0.37400285  1.1845522 ]
[-0.40006916  0.37365253  1.18352

In [ ]:
df = pd.read_csv('data/age_video1.csv')
RT = df['RT'].to_numpy()
R = df['R'].to_numpy()
df.head()
# Rename the output likelihood_output.txt and estimate_output.txt in the data folder before running the code.
res3 = minimize(objective_function, initial_guess, method='Nelder-Mead', options={'maxiter': 150}, callback=callback)


In [ ]:
df = pd.read_csv('data/age_video2.csv')
RT = df['RT'].to_numpy()
R = df['R'].to_numpy()
df.head()
# Rename the output likelihood_output.txt and estimate_output.txt in the data folder before running the code.
res4 = minimize(objective_function, initial_guess, method='Nelder-Mead', options={'maxiter': 150}, callback=callback)

In [ ]:
df = pd.read_csv('data/gender_text.csv')
RT = df['RT'].to_numpy()
R = df['R'].to_numpy()
df.head()
# Rename the output likelihood_output.txt and estimate_output.txt in the data folder before running the code.
res5 = minimize(objective_function, initial_guess, method='Nelder-Mead', options={'maxiter': 150}, callback=callback)

In [ ]:
df = pd.read_csv('data/gender_video.csv')
RT = df['RT'].to_numpy()
R = df['R'].to_numpy()
df.head()
# Rename the output likelihood_output.txt and estimate_output.txt in the data folder before running the code.
res6 = minimize(objective_function, initial_guess, method='Nelder-Mead', options={'maxiter': 150}, callback=callback)